In [ ]:
import pandas as pd
import boto3
import json
import configparser

## Start by setting up the configuration variables. 

In [ ]:
config = configparser.ConfigParser()
config.read_file(open('credentials/dwh.cfg'))

KEY                    = config.get('AWS','KEY')
SECRET                 = config.get('AWS','SECRET')

DWH_CLUSTER_TYPE       = config.get("DWH","DWH_CLUSTER_TYPE")
DWH_NUM_NODES          = config.get("DWH","DWH_NUM_NODES")
DWH_NODE_TYPE          = config.get("DWH","DWH_NODE_TYPE")

DWH_CLUSTER_IDENTIFIER = config.get("DWH","DWH_CLUSTER_IDENTIFIER")
DWH_DB                 = config.get("DWH","DWH_DB")
DWH_DB_USER            = config.get("DWH","DWH_DB_USER")
DWH_DB_PASSWORD        = config.get("DWH","DWH_DB_PASSWORD")
DWH_PORT               = config.get("DWH","DWH_PORT")

DWH_IAM_ROLE_NAME      = config.get("DWH", "DWH_IAM_ROLE_NAME")

## Then create the clients for EC2, S3, IAM and Redshift

In [ ]:
ec2 = boto3.resource('ec2',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET)

s3 = boto3.resource('s3',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET)

iam = boto3.client('iam',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET)

redshift = boto3.client('redshift',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET)


## We create an IAM role that allow redshift to read from an S3 bucket

In [ ]:
try:
    print('1.1 Creating a new IAM Role')
    dwhRole = iam.create_role(
        Path='/',
        RoleName=DWH_IAM_ROLE_NAME,
        Description="Allows Redshift clusters to call AWS services on your behalf",
        AssumeRolePolicyDocument=json.dumps(
                    {'Statement': [{'Action': 'sts:AssumeRole',
                                    'Effect': 'Allow',
                                    'Principal': {'Service': 'redshift.amazonaws.com'}}],
                     'Version': '2012-10-17'})
    )
    

except Exception as e:
    print(e)

In [ ]:
print('1.2 Attaching Policy')
iam.attach_role_policy(RoleName=DWH_IAM_ROLE_NAME,
                      PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
                      )['ResponseMetadata']['HTTPStatusCode']

In [ ]:
print('1.3 Get the IAM role ARN')
roleArn = iam.get_role(RoleName=DWH_IAM_ROLE_NAME)['Role']['Arn']

## This following cell will create a redshift cluster, Careful as this will incur in some charges in your AWS account
* Run only the first cell to create the cluster, then the second and third will allow you to check the status.
ONLY proceed when it becomes available.

In [ ]:
try:
    response = redshift.create_cluster(        
        # TODO: add parameters for hardware
        ClusterType=DWH_CLUSTER_TYPE,
        NodeType=DWH_NODE_TYPE,
        NumberOfNodes=int(DWH_NUM_NODES),

        # TODO: add parameters for identifiers & credentials
        DBName=DWH_DB,
        ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,
        MasterUsername=DWH_DB_USER,
        MasterUserPassword=DWH_DB_PASSWORD,
                
        # TODO: add parameter for role (to allow s3 access)
        IamRoles=[roleArn] 
    )
except Exception as e:
    print(e)

In [ ]:
#This function will display the status of our cluster
def prettyRedshiftProps(props):
    pd.set_option('display.max_colwidth', None)
    keysToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "NumberOfNodes", 'VpcId']
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x, columns=["Key", "Value"])

In [ ]:
myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

** RUN THIS ONLY AFTER THE STATUS BECOMES AVAILABLE **

In [ ]:
DWH_ENDPOINT = myClusterProps['Endpoint']['Address']
DWH_ROLE_ARN = myClusterProps['IamRoles'][0]['IamRoleArn']
print("DWH_ENDPOINT :: ", DWH_ENDPOINT)
print("DWH_ROLE_ARN :: ", DWH_ROLE_ARN)

## Opening a TCP port to access the cluster endpoint 
The port might already exist, and it would throw back an error if it's the case

In [ ]:
try:
    vpc = ec2.Vpc(id=myClusterProps['VpcId'])
    defaultSg = list(vpc.security_groups.all())[0]
    print(defaultSg)
    
    defaultSg.authorize_ingress(
        GroupName= defaultSg.group_name, 
        CidrIp='0.0.0.0/0',  
        IpProtocol='TCP', 
        FromPort=int(DWH_PORT),
        ToPort=int(DWH_PORT)
    )
except Exception as e:
    print(e)

## Checking if we can connect to the cluster

In [ ]:
%reload_ext sql

In [ ]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

In [ ]:
conn_string="postgresql://{}:{}@{}:{}/{}".format(DWH_DB_USER, DWH_DB_PASSWORD, DWH_ENDPOINT, DWH_PORT,DWH_DB)
print(conn_string)
%sql $conn_string

## Test that we can create,populate and query against a table '

-> If this works, we can proceed to run create_tables.py to build the backbone of our schema in the cluster.  
-> Be careful to not go ahead with cleaning resources (end of this notebook) before the pipelines finish to run.    

In [ ]:
%sql CREATE TABLE IF NOT EXISTS test (col1 TEXT, col2 TEXT, col3 INT);
%sql INSERT INTO test (col1,col2,col3) VALUES ('hello','world',55),('another','world',34),('bananas','pyjamas',69);
%sql SELECT * FROM test ORDER BY col1 desc ;


In [ ]:
# And cleanup the table
%sql DROP TABLE test

## To check if there were any errors creating/inserting data in the cluster, run this query:

In [ ]:
%sql SELECT * FROM quadrant_table LIMIT 5;

In [ ]:
%sql SELECT * FROM stl_load_errors ORDER BY starttime desc ;

In [ ]:
%%sql
TRUNCATE TABLE quadrant_table;
COPY quadrant_table
FROM 's3://alexskp-capstone/bucket_data/quadrant_table.csv'
IAM_ROLE 'arn:aws:iam::261476836151:role/dwhRole'
REGION 'us-west-2'
csv
IGNOREHEADER 1;

## Clean resources: This will delete the cluster and the IAM role. 
1. Run only the first cell to delete the cluster. Very important if you don't want to incur in more charges 
2. Wait until it's deleted checking the status by running the second cell 
3. Run the last cell to delete the user role

In [ ]:
#### CAREFUL!!
#-- Uncomment & run to delete the created resources
redshift.delete_cluster( ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,  SkipFinalClusterSnapshot=True)
#### CAREFUL!!

In [ ]:
# RUN THIS TO CHECK THE STATUS OF THE CLUSTER UNTIL IT'S NOT FOUND ANYMORE

myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

In [ ]:
#### CAREFUL!!
#-- Uncomment & run to delete the created resources
iam.detach_role_policy(RoleName=DWH_IAM_ROLE_NAME, PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess")
iam.delete_role(RoleName=DWH_IAM_ROLE_NAME)
#### CAREFUL!!